# カスタムLLMエージェント（ChatModel付き）

このノートブックでは、ChatModelをベースにした独自のカスタムエージェントの作成方法を説明します。

LLM（大規模言語モデル）チャットエージェントは3つの部分で構成されています。

- PromptTemplate：言語モデルに何をするか指示するのに使用できるプロンプトテンプレートです
- ChatModel：エージェントを駆動する言語モデルです
- `stop`シーケンス：この文字列が見つかったらすぐにLLMの生成を停止するように指示します
- OutputParser：この部分では、LLMが出力したものをどのようにエージェントのアクションまたは終了オブジェクトに変換するかを決定します

LLMAgentはAgentExecutorで使用されます。AgentExecutorは主に以下のループで構成されていると考えることができます。
1. ユーザーの入力や前のステップをエージェント（本件ではLLMAgent）に渡します
2. エージェントが`AgentFinish`を返す場合は、それを直接ユーザーに返します
3. エージェントが`AgentAction`を返す場合は、それを使ってツールを呼び出し、`Observation`を取得します
4. `AgentAction`と`Observation`をエージェントに返すことを繰り返し、`AgentFinish`が発行されるまで行います。

`AgentAction`は、「action」と「action_input」で構成されるレスポンスです。「action」は使用するツールを指し、「action_input」はそのツールへの入力を指します。「log」というコンテキストも提供され、ログやトレースなどに使用できます。

`AgentFinish`は、最終的にユーザーに返すメッセージが含まれているレスポンスです。これはエージェントの実行を終了するために使用されるべきです。

このノートブックでは、カスタムLLMエージェントの作成方法について説明します。

## 環境を設定する

必要なインポートなどを行ってください。

In [1]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

## ツールのセットアップ

エージェントが使用したいツールをセットアップします。これはプロンプトに記入する必要があるかもしれません（そのため、エージェントがこれらのツールを使用することを知っています）。

In [2]:
# エージェントがユーザーの問い合わせに答えるために使用できるツールを定義する。
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="時事に即した質問に答えたいときに便利です。"
    )
]

## プロンプトテンプレート

このテンプレートは、エージェントに何をすべきか指示します。一般的に、テンプレートは以下の要素を含むべきです。

- `tools`：エージェントが利用できるツールとそれらをどのように、いつ呼び出すか。
- `intermediate_steps`：これらは、以前の（`AgentAction`、`Observation`）ペアのタプルです。これらは通常、モデルに直接渡されるわけではなく、プロンプトテンプレートがそれらを特定の方法でフォーマットします。
- `input`：一般的なユーザー入力

In [13]:
# Set up the base template
template = """可能な限り目的を達成してください。解答は日本語でして下さい。あなたは以下のツールにアクセスできます:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""

In [14]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [15]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## 出力パーサー

出力パーサーは、LLMの出力を`AgentAction`と`AgentFinish`に解析する役割を担っています。これは通常、使用されるプロンプトに大きく依存します。

ここでは、再試行の実行や空白の処理など、解析方法を変更することができます。

In [16]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [17]:
output_parser = CustomOutputParser()

## LLMの設定

使用したいLLMを選んでください！

In [25]:
llm = ChatOpenAI(temperature=0.2)

## ストップシーケンスの定義

これは、LLMにいつ生成を停止するかを伝えるために重要です。

これは、使用しているプロンプトやモデルに大きく依存します。一般的には、プロンプトで`Observation`の開始を示すために使用するトークンがこれになります（そうでない場合、LLMが観測を幻覚するかもしれません）。

## エージェントの設定

これで、すべてを組み合わせてエージェントを設定することができます。

In [26]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [27]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

## エージェントの使用

これで使えます！

In [28]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [30]:
agent_executor.run("Dead by Daylightの最新のアップデートはいつでしたか？")



> Entering new AgentExecutor chain...
Thought: Dead by Daylightの最新のアップデートについては、私は情報を持っていません。
Action: Search
Action Input: "Dead by Daylight 最新アップデート"

Observation:DbD(デッドバイデイライト)のアップデート最新情報を紹介！アプデはいつ実装するのかやパッチノートの内容、過去のアプデ情報についても記載している ...
最新のアップデートはまだ実装されていないようです。
Final Answer: 最新のDead by Daylightのアップデートはまだ実装されていない。

> Finished chain.


'最新のDead by Daylightのアップデートはまだ実装されていない。'